<a href="https://colab.research.google.com/github/jeff0908/stock_predict/blob/main/stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math

In [ ]:
import pandas_datareader as web
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
#get stock info
df = web.DataReader('TSLA', data_source='yahoo', start='2020-01-01', end='2024-03-01')
#show data
df

TypeError: string indices must be integers